In [1]:
import mindspore
from mindspore import nn
from mindspore.dataset.transforms import PadEnd
from mindspore.dataset.text import Lookup
from mindspore.common.initializer import initializer, XavierNormal, Orthogonal

import mindnlp
from mindnlp.modules import Glove
from mindnlp.transforms import BasicTokenizer
from mindnlp.scoring.metrics import accuracy

from tqdm import tqdm

[ERROR] ME(30337:140612048580800,MainProcess):2023-03-07-11:19:17.127.5 [mindspore/run_check/_check_version.py:226] Cuda ['10.1', '11.1', '11.6'] version(libcu*.so need by mindspore-gpu) is not found, please confirm that the path of cuda is set to the env LD_LIBRARY_PATH, or check whether the CUDA version in wheel package and the CUDA runtime in current device matches, please refer to the installation guidelines: https://www.mindspore.cn/install
[ERROR] ME(30337:140612048580800,MainProcess):2023-03-07-11:19:17.272.39 [mindspore/run_check/_check_version.py:226] Cuda ['10.1', '11.1', '11.6'] version(libcudnn*.so need by mindspore-gpu) is not found, please confirm that the path of cuda is set to the env LD_LIBRARY_PATH, or check whether the CUDA version in wheel package and the CUDA runtime in current device matches, please refer to the installation guidelines: https://www.mindspore.cn/install


In [2]:
imdb_train, imdb_test = mindnlp.load_dataset('imdb', split=['train', 'test'], shuffle=True)

In [3]:
# load embedding and vocab
embedding, vocab = Glove.from_pretrained('6B', 100, special_tokens=["<unk>", "<pad>"])

In [4]:
tokenizer = BasicTokenizer(lower_case=True)

In [5]:
lookup_op = Lookup(vocab, unknown_token='<unk>')

In [6]:
max_length = 256
pad_op = PadEnd([max_length], pad_value=vocab.tokens_to_ids('<pad>'))

In [7]:
imdb_train = imdb_train.map([tokenizer, lookup_op, pad_op], 'text')
imdb_test = imdb_test.map([tokenizer, lookup_op, pad_op], 'text')

In [8]:
batch_size = 64

imdb_train = imdb_train.batch(batch_size)
imdb_test = imdb_test.batch(batch_size)

In [9]:
imdb_train, imdb_valid = imdb_train.split([0.7, 0.3])

[WARNING] ME(30337:140612048580800,MainProcess):2023-03-07-11:19:50.253.869 [mindspore/dataset/engine/datasets.py:1142] Dataset is shuffled before split.


In [10]:
import math
import mindspore as ms
import mindspore.nn as nn
import mindspore.ops as ops
from mindspore.common.initializer import Uniform, HeUniform

class RNN(nn.Cell):
    def __init__(self, embedding, hidden_dim, output_dim, n_layers,
                 bidirectional, pad_idx):
        super().__init__()
        embedding_dim = embedding._embed_dim
        self.embedding = embedding
        self.rnn = nn.LSTM(embedding_dim,
                           hidden_dim,
                           num_layers=n_layers,
                           bidirectional=bidirectional,
                           batch_first=True,
                           dropout=0.5)
        self.fc = nn.Dense(hidden_dim * 2, output_dim)

    def construct(self, inputs):
        embedded = self.embedding(inputs)
        _, (hidden, _) = self.rnn(embedded)
        hidden = ops.concat((hidden[-2, :, :], hidden[-1, :, :]), axis=1)
        output = self.fc(hidden)
        return output

In [11]:
hidden_size = 256
output_size = 2
num_layers = 2
bidirectional = True
lr = 5e-4
pad_idx = vocab.tokens_to_ids('<pad>')

model = RNN(embedding, hidden_size, output_size, num_layers, bidirectional, pad_idx)
loss_fn = nn.CrossEntropyLoss()
optimizer = nn.Adam(model.trainable_params(), learning_rate=lr)

In [12]:
def initialize_weights(m):
    if isinstance(m, nn.Dense):
        m.weight.set_data(initializer(XavierNormal(), m.weight.shape, m.weight.dtype))
        m.bias.set_data(initializer('zeros', m.bias.shape, m.bias.dtype))
    elif isinstance(m, nn.LSTM):
        for name, param in m.parameters_and_names():
            if 'bias' in name:
                param.set_data(initializer('zeros', param.shape, param.dtype))
            elif 'weight' in name:
                param.set_data(initializer(Orthogonal(), param.shape, param.dtype))

In [13]:
model.apply(initialize_weights)

RNN<
  (embedding): Glove<
    (dropout_layer): Dropout<keep_prob=1.0>
    >
  (rnn): LSTM<
    (rnn): _DynamicLSTMCPUGPU<>
    (dropout_op): Dropout<p=0.5>
    >
  (fc): Dense<input_channels=512, output_channels=2, has_bias=True>
  >

In [14]:
def forward_fn(data, label):
    logits = model(data)
    loss = loss_fn(logits, label)
    return loss

grad_fn = ms.value_and_grad(forward_fn, None, optimizer.parameters)

def train_step(data, label):
    loss, grads = grad_fn(data, label)
    optimizer(grads)
    return loss

def train_one_epoch(model, train_dataset, epoch=0):
    model.set_train()
    total = train_dataset.get_dataset_size()
    loss_total = 0
    step_total = 0
    with tqdm(total=total) as t:
        t.set_description('Epoch %i' % epoch)
        for data, label in train_dataset.create_tuple_iterator():
            loss = train_step(data, label.astype(mindspore.int32))
            loss_total += loss.asnumpy()
            step_total += 1
            t.set_postfix(loss=loss_total/step_total)
            t.update(1)


In [15]:
def evaluate(model, test_dataset, criterion, epoch=0):
    total = test_dataset.get_dataset_size()
    epoch_loss = 0
    epoch_acc = 0
    step_total = 0
    model.set_train(False)

    with tqdm(total=total) as t:
        t.set_description('Epoch %i' % epoch)
        for i in test_dataset.create_tuple_iterator():
            predictions = model(i[0])
            loss = criterion(predictions, i[1].astype(mindspore.int32))
            epoch_loss += loss.asnumpy()

            acc = accuracy(predictions, i[1])
            epoch_acc += acc

            step_total += 1
            t.set_postfix(loss=epoch_loss/step_total, acc=epoch_acc/step_total)
            t.update(1)

    return epoch_loss / total

In [16]:
num_epochs = 5
best_valid_loss = float('inf')

for epoch in range(num_epochs):
    train_one_epoch(model, imdb_train, epoch)
    valid_loss = evaluate(model, imdb_valid, loss_fn, epoch)

    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        ms.save_checkpoint(model, './sentiment_analysis.ckpt')

Epoch 4: 100%|████████████████████████████████████████████████████████████████████████████████████| 117/117 [00:07<00:00, 15.04it/s, acc=0.874, loss=0.318]
